In [1]:
from bs4 import BeautifulSoup
import requests

In [14]:
def download_data(url, proxy=None):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0",
        "Accept": "*/*",
        "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br"}

    s = requests.Session()
    page = s.get(url,
                 headers=headers,
                 proxies={"http": proxy, "https": proxy}, stream=True
                )
    # Stream=True => In chunked transfer encoding, the data stream is divided into a series of non-overlapping "chunks". The chunks are sent out independently of one another by the server. Hope this answers the question.
#     if page.status_code == 429:
#         print("Blocked due to many requests")
#     if page.status_code == 403:
#         raise Exception("Url is forbidden")

    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [15]:
page = download_data("https://www.youtube.com/watch?v=8kEOoLomYzA")

In [446]:
print(page.prettify())

NameError: name 'page' is not defined

# Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
options = Options()
options.add_argument("--headless")

In [3]:
service = Service(executable_path="./geckodriver")
driver = webdriver.Firefox(service=service, options=None)

In [4]:
driver.get("https://www.youtube.com/watch?v=8kEOoLomYzA")

# get all sentences 

In [283]:
#url = "ytd-menu-service-item-renderer.style-scope:nth-child(2) > tp-yt-paper-item:nth-child(1) > yt-formatted-string:nth-child(2)"
url = "ytd-menu-renderer.ytd-video-primary-info-renderer > yt-icon-button:nth-child(3) > button:nth-child(1)"
button = driver.find_element_by_css_selector(url)

/tmp/ipykernel_1120332/3729645192.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  button = driver.find_element_by_css_selector(url)


In [284]:
button.click()

hidden buttion

In [285]:
url = "ytd-menu-service-item-renderer.style-scope:nth-child(2) > tp-yt-paper-item:nth-child(1) > yt-formatted-string:nth-child(2)"
button = driver.find_element_by_css_selector(url)

/tmp/ipykernel_1120332/876996630.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  button = driver.find_element_by_css_selector(url)


In [286]:
button.click()

In [287]:
css_selector = "html body ytd-app div#content.style-scope.ytd-app ytd-page-manager#page-manager.style-scope.ytd-app ytd-watch-flexy.style-scope.ytd-page-manager.hide-skeleton div#columns.style-scope.ytd-watch-flexy div#secondary.style-scope.ytd-watch-flexy div#secondary-inner.style-scope.ytd-watch-flexy div#panels.style-scope.ytd-watch-flexy ytd-engagement-panel-section-list-renderer.style-scope.ytd-watch-flexy"
transcript = driver.find_element_by_css_selector(css_selector)

/tmp/ipykernel_1120332/1986423723.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  transcript = driver.find_element_by_css_selector(css_selector)


In [288]:
#xpath = "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/div[1]/ytd-engagement-panel-section-list-renderer[4]/div[2]/ytd-transcript-renderer/div[2]/ytd-transcript-search-panel-renderer/div[2]/ytd-transcript-segment-list-renderer/div[1]/ytd-transcript-segment-renderer"
# short version
xpath = "//div[contains(@class, 'segment style-scope ytd-transcript-segment-renderer')]"
teams = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, xpath)))

In [289]:
all_text = []
for x in teams:
    all_text.append(x.text.split('\n')[1])
    #break

In [290]:
#all_text = " ".join(all_text)

In [291]:
import re
for text in all_text:
    res = [(x.group(0), x.span()) for x in list(re.finditer("\s(\w+)?ху(\w*)?\s", text))]
    if res:
        print(res)

[(' хуйню ', (20, 27))]
[(' хуйню ', (6, 13))]
[(' нахуй ', (14, 21))]
[(' хуй ', (19, 24))]
[(' нюху ', (13, 19))]
[(' нахуй ', (12, 19))]
[(' хуячить ', (19, 28))]
[(' хуйню ', (16, 23))]
[(' нахуя ', (21, 28))]
[(' нахуй ', (9, 16))]
[(' нахуй ', (6, 13))]
[(' ахуенно ', (26, 35))]
[(' худший ', (7, 15))]
[(' хуже ', (24, 30))]
[(' хуйнёй ', (26, 34))]


# single example

In [102]:
xpath = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/div[1]/ytd-engagement-panel-section-list-renderer[4]/div[2]/ytd-transcript-renderer/div[2]/ytd-transcript-search-panel-renderer/div[2]/ytd-transcript-segment-list-renderer/div[1]/ytd-transcript-segment-renderer[1]/div'
descr = transcript.find_element_by_xpath(xpath)

/tmp/ipykernel_1120332/1599345750.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  descr = transcript.find_element_by_xpath(xpath)


In [106]:
descr.text

'0:00\nпринципе можно конечно'

# get all videos from the channel 

In [8]:
driver.get("https://www.youtube.com/c/MadHighlights/videos")

In [83]:
# test
xpath = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer'
#xpath = '//*[@id="dismissible"]'
#xpath = '//div[contains(@class,"stroke style-scope yt-interaction")]'
video_preview = driver.find_elements_by_xpath(xpath)

/tmp/ipykernel_13409/915433493.py:5: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  video_preview = driver.find_elements_by_xpath(xpath)


In [92]:
len(video_preview)

30

In [97]:
for x in video_preview:
    link = x.find_elements_by_id('video-title')
    assert len(link) == 1, "links number are not equal to 1 for video"
    link = link[0].get_attribute('href')
    print(x.text, link)

/tmp/ipykernel_13409/3469597454.py:2: DeprecationWarning: find_elements_by_id is deprecated. Please use find_elements(by=By.ID, value=id_) instead
  link = x.find_elements_by_id('video-title')


45:23
Мэддисон катает шары в Elden Ring
33K views
6 hours ago https://www.youtube.com/watch?v=0iydsNVpqX0
29:15
Мэддисона забуллил взбесившийся волк в Elden Ring
140K views
6 days ago https://www.youtube.com/watch?v=8kEOoLomYzA
35:19
Мэддисон и очередной дед в Elden Ring
142K views
7 days ago https://www.youtube.com/watch?v=GlfZmcvRR7k
30:22
Мэддисон и дед номер два в Elden ring
141K views
9 days ago https://www.youtube.com/watch?v=aKerGDE5OTM
34:43
Мэддисон и его первый дед в Elden Ring
186K views
10 days ago https://www.youtube.com/watch?v=xWlj29Aa20s
39:24
Мэддисон и Welovegames отмечают десятилетие Warface
139K views
2 weeks ago https://www.youtube.com/watch?v=hDk5ulqy4V0
38:16
Мэддисон играет в Макса Пейна НА РУЛЕ
282K views
4 weeks ago https://www.youtube.com/watch?v=4CsK9a-2Y38
1:15:09
Мэддисон играет в Lost Ark и прощается с Твичем
193K views
1 month ago https://www.youtube.com/watch?v=wWTp4TkPF2s
42:01
Мэддисон нашел персонажа которого озвучил в Lost Ark
154K views
1 month ago

# Scroll to get all videos

In [503]:
from selenium.webdriver.common.action_chains import ActionChains

In [504]:
    from selenium.webdriver.common.keys import Keys

In [505]:
driver.get("https://www.youtube.com/c/MadHighlights/videos")

In [506]:
import time

In [508]:
xpath = '//*[@id="contents"]'
element = driver.find_element_by_xpath(xpath) # the element you want to scroll to 

/tmp/ipykernel_1120332/3837978472.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element = driver.find_element_by_xpath(xpath) # the element you want to scroll to


In [509]:
prev_page_pos = 0
while True:
    #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    driver.find_element_by_tag_name('body').send_keys(Keys.END) # triggers AJAX request to load more users. observed that loading 10 users at a time.
    new_page_pos = str(driver.execute_script('return window.pageYOffset;'))
    print(prev_page_pos, new_page_pos)
    if prev_page_pos == new_page_pos:
        break
    prev_page_pos = new_page_pos
    time.sleep(1)

/tmp/ipykernel_1120332/1628361123.py:4: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.END) # triggers AJAX request to load more users. observed that loading 10 users at a time.


0 0
0 707
707 1787
1787 2867
2867 3947
3947 5027
5027 6107
6107 7187
7187 8267
8267 9347
9347 10427
10427 11507
11507 12587
12587 13667
13667 14747
14747 15827
15827 16907
16907 17987
17987 19067
19067 20189
20189 21227
21227 22307
22307 23387
23387 24467
24467 25547
25547 26627
26627 27707
27707 28787
28787 29867
29867 30947
30947 32052
32052 33107
33107 34187
34187 35451
35451 35855
35855 35855


# close

In [315]:
driver.close() #?
driver.quit()

InvalidSessionIdException: Message: Tried to run command without establishing a connection
